In [14]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1] - b[1], c[0]-b[0]) - np.arctan2(a[1] - b[1], a[0]-b[0]) # arctan2 : 탄젠트 값 반환함  
    angle = np.abs(radians * 180.0/np.pi) # abs 절댓값
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

def Armpit(a):
    if a>70 and a<100:
        return 1
    else:
        if a > 100:
            return "+1"
        elif a < 70:
            return "-1"

cap = cv2.VideoCapture(0)
with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("카메라를 찾을 수 없습니다.")
            continue

        # 필요에 따라 성능 향상을 위해 이미지 작성을 불가능함으로 기본 설정합니다.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # 포즈 주석을 이미지 위에 그림
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        try:
            # Get coordinates
            landmarks = results.pose_landmarks.landmark
            
            L_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            L_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            L_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            L_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            L_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y]
            
            R_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            R_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            R_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            R_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            R_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y]
            
            # Calculate angle
            L_wrist_angle = calculate_angle(L_shoulder, L_elbow, L_wrist)
            R_wrist_angle = calculate_angle(R_shoulder, R_elbow, R_wrist)
            L_hip_angle = calculate_angle(L_shoulder,L_hip,L_knee)
            R_hip_angle = calculate_angle(R_shoulder,R_hip,R_knee)
            L_armpit_angle = calculate_angle(L_elbow, L_shoulder, L_hip)
            R_armpit_angle = calculate_angle(R_elbow, R_shoulder, R_hip)
            
            
            # input status text
            status = "I don't know"
            hip_status = " "
            position = " "
        
            L_arm = Armpit(L_armpit_angle)
            R_arm = Armpit(R_armpit_angle)
            
            if (Armpit(L_arm) == 1) and (Armpit(R_arm) == 1):
                position = "good posture"
            elif L_arm == "+1" or R_arm == "+1":
                position = "put your arms back a little"
            elif L_arm == "-1" or R_arm == "-1":
                position = "put your arms forward a little"
            else:
                position = "I don't know"
                
            
                             
            # L_wrist_angle이 90정도가 굽힘 
            # if L_wrist_angle 
            
            # if L_angle > R_angle:
            #     if (L_angle - R_angle) < 30:
            #         status = "Good push up"
            #     else:
            #         status = "No, fix your posture"
            # else:
            #     if (R_angle - L_angle) < 30:
            #         status = "Good push up"
            #     else:
            #         status = "No, fix your posture"
                    
            if(L_hip_angle < 190 and L_hip_angle > 170) and (R_hip_angle < 190 and L_hip_angle < 170):
                hip_status = " "
            else:
                hip_status = "fix your hip"
                    
            cv2.putText(image, str(status), (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,255,0), 2, cv2.LINE_AA)
            cv2.putText(image, str(hip_status), (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,255,0), 2, cv2.LINE_AA)


            cv2.putText(image,str(int(L_wrist_angle)),
                            tuple(np.multiply(L_elbow, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(int (R_wrist_angle)),
                            tuple(np.multiply(R_elbow, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                    )
            cv2.putText(image, str(int (L_armpit_angle)),
                            tuple(np.multiply(L_shoulder, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                    )
        except:
            pass
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color = (245,117,66), thickness = 2, circle_radius = 2),
            mp_drawing.DrawingSpec(color = (245,66,230), thickness = 2, circle_radius = 2)
        )
        
        # 보기 편하게 이미지를 좌우 반전합니다.
#         cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == 27: # ESC
            break
    cap.release()
    cv2.destroyAllWindows()

60.150173135725915
38.03658397074301
37.028702887696205
43.981332884625274
45.318778077940394
48.80410627238104
47.31247640492245
46.97529131210469
60.3568694984864
50.58496614021262
70.21409284973518
79.32324717322076
129.94196107627377
143.34010940657345
148.03094308858857
154.31468780177443
156.79800144830813
158.70776637454966
157.75921205415494
157.5316857267073
158.0454841545785
157.5097434032119
157.34652455639335
157.04781547108152
156.85982888844072
156.7012729229495
156.77519441838004
156.97906424082356
157.00991334550767
156.6672005310248
156.24116149360052
154.73481514036044
152.2317212305809
151.79559455555633
150.40830747996023
146.6570722317792
144.69321090234595
145.4218245191685
144.0922004066936
142.03254164344577
140.4009327922734
139.64705471026443
138.23306846260718
136.7845120983812
129.1205188690608
128.95230146290285
129.35171999867407
119.5077344942012
115.37675701618312
112.89003859238208
112.69286135193244
113.77806414775088
100.03484393474379
95.092172521546

In [10]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1] - b[1], c[0]-b[0]) - np.arctan2(a[1] - b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [8]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]

In [6]:
shoulder

[0.7528208494186401, 1.360909342765808]

In [11]:
calculate_angle(shoulder, elbow, wrist)

61.24743699490333